Need to include referencing 

# Bank Customer Churn Prediction

In this part of the report we look at customer banking data to discover the reasons customers terminate their accounts. Churn prediction is how we identify the likelihood of this. This predictive analyis involves examining various customer behaviours, demographic data and products to spot potential churners before they leave.

## 1. Read Data and Import Necessary Functions

## Import Functions

In [ ]:
# For data wrangling 
import numpy as np
import pandas as pd

# For visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

## Reading the Data

Here we found that storing the bank churn csv file in a google drive was the simplest method. From getting the data in Kaggle, there is an intricate method to automate the download of data, involving authentication requirements. 

In [ ]:
# Direct link to the CSV file on Google Drive
csv_url = 'https://drive.google.com/uc?id=14HTdhAmJd_U3mJDnwLHPbu6pvh-Yv0vu'

# Read the CSV file from the URL
df = pd.read_csv(csv_url)

df.shape

The DataFrame has 10000 rows with 14 attributes. We continue to identify what attributes will be necessary and what data manipulation needs to be carried out before Exploratory analysis and prediction modelling can be done. 


## Initital Exploration

The next commands check the data. 